In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB


df = pd.read_csv(r'C:\Dataset\kddcup99.csv')

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [2]:
df['protocol_type'] = pd.to_numeric(df['protocol_type'], errors='coerce').fillna(0, downcast="float")
df['service'] = pd.to_numeric(df['service'], errors='coerce').fillna(0, downcast="float")
df['flag'] = pd.to_numeric(df['flag'], errors='coerce').fillna(0, downcast="float")

In [3]:
new_label = df['label'].copy()

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [5]:
X = df.drop(['label'], axis=1)
y = df['label']

In [6]:
X = X.astype('float32')

In [7]:
print(X.shape, y.shape)

(494020, 41) (494020,)


In [8]:
lreg = LinearRegression()
sfs1 = sfs(lreg, k_features=5, forward=True, verbose=2, scoring='neg_mean_squared_error')
sfs1 = sfs1.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:    4.3s finished

[2023-01-30 20:58:13] Features: 1/5 -- score: -1.7215644328847575[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   10.2s finished

[2023-01-30 20:58:23] Features: 2/5 -- score: -1.4243103511899202[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:   11.0s finished

[2023-01-30 20:58:34] Features: 3/5 -- score: -1.2531682913194684[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [9]:
feat_names = list(sfs1.k_feature_names_)
print(feat_names)

print(df[feat_names].shape, df.shape)

['wrong_fragment', 'count', 'srv_count', 'same_srv_rate', 'diff_srv_rate']
(494020, 5) (494020, 42)


In [10]:
new_df = df[feat_names]
new_df['label'] = new_label

new_df.head()

C:\Users\KAYRA\AppData\Local\Temp\ipykernel_11868\326514612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['label'] = new_label


,wrong_fragment,count,srv_count,same_srv_rate,diff_srv_rate,label
0,0,8,8,1.0,0.0,normal
1,0,8,8,1.0,0.0,normal
2,0,8,8,1.0,0.0,normal
3,0,6,6,1.0,0.0,normal
4,0,6,6,1.0,0.0,normal


In [11]:
X = new_df.drop(['label'],axis=1)
y = new_df['label']

In [12]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [13]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8516962875996923